<a href="https://colab.research.google.com/github/khaerensml6/mlcon-workshop/blob/main/ControlNet_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Controlnet

First we make sure a GPU is connected to our runtime. If this is not the case: in the upper right corner you can change the runtime, and select one with a T4 GPU

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))
    print("Make sure to select T4 gpu (upper-right corner > change runtime type > T4)")



Found GPU at: /device:GPU:0


Next, we download a convenient helper script from huggingface diffusers for training controlnet

**Ignore the warning about restarting the runtime!**

In [4]:
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .
!pip install bitsandbytes
%cd examples/controlnet
!pip install -r requirements.txt


Cloning into 'diffusers'...
remote: Enumerating objects: 49850, done.
remote: Counting objects: 100% (49849/49849), done.
remote: Compressing objects: 100% (9840/9840), done.
^C
[Errno 2] No such file or directory: 'diffusers'
/content/diffusers/examples/controlnet
Obtaining file:///content/diffusers/examples/controlnet
ERROR: file:///content/diffusers/examples/controlnet does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/bas

We create a basic **accelerate** config



In [5]:
from accelerate.utils import write_basic_config
write_basic_config()

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


False


## Log into Huggingface to access the dataset you've just created


In [6]:
from huggingface_hub import notebook_login
notebook_login()

Here you need to fill in some environment variables to point to your dataset.

Also the column names need to be set correctly.

In [13]:
import os

#TODO complete the environment variables

os.environ["MODEL_DIR"] = "runwayml/stable-diffusion-v1-5"
os.environ["OUTPUT_DIR"] = "/content/trained_model"
os.environ["DATASET_NAME"] =  # "fusing/fill50k"
os.environ["IMAGE_COLUMN"] =  #"image"
os.environ["CONDITIONING_IMAGE_COLUMN"] = #"conditioning_image"
os.environ["CAPTIONING_COLUMN"] = #"text"



Below command will run the training.

In [14]:
!accelerate launch train_controlnet.py \
 --pretrained_model_name_or_path=$MODEL_DIR \
 --output_dir=$OUTPUT_DIR \
 --dataset_name=$DATASET_NAME \
 --resolution=512 \
 --learning_rate=1e-5 \
 --image_column=$IMAGE_COLUMN\
 --conditioning_image_column=$CONDITIONING_IMAGE_COLUMN\
 --caption_column=$CAPTIONING_COLUMN\
# --validation_image "./conditioning_image_1.png" "./conditioning_image_2.png" \
# --validation_prompt "red circle with blue background" "cyan circle with brown floral background" \
 --train_batch_size=1 \
 --gradient_accumulation_steps=4 \
 --gradient_checkpointing \
 --use_8bit_adam

2023-11-30 03:55:49.786509: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 03:55:49.786565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 03:55:49.786613: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 03:55:51.037844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/30/2023 03:55:51 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of t

KeyboardInterrupt: ignored